In [8]:
import os
import sys
import math
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [9]:
df_path = "data/diabetic_data.csv"

try:
    df = pd.read_csv(df_path)
    print("Dataset loaded from:", df_path)
except Exception as e:
    raise RuntimeError(f"Could not load {df_path}. Error: {e}")

Dataset loaded from: data/diabetic_data.csv


In [10]:
# 1) Inspect unique values of target
print("Unique values in 'readmitted':", df['readmitted'].unique())

# Map readmitted to binary: '<30' -> 1, else 0
df['readmit_30d'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

# 2) Drop identifier columns not useful for prediction
for col in ['encounter_id', 'patient_nbr']:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)
        print(f"Dropped column: {col}")

# 3) Basic missingness overview
missing_summary = df.isnull().sum().sort_values(ascending=False)
display(missing_summary[missing_summary > 0].head(30))

for col in df.columns:
    if df[col].dtype == object and (df[col] == '?').any():
        df[col].replace('?', np.nan, inplace=True)
        print(f"Replaced '?' with NaN in {col}")

# Recompute missing summary
display(df.isnull().sum().sort_values(ascending=False).head(30))

# 4) Drop columns with too many missing values
missing_threshold = 0.5  # drop columns with >50% missing
n_rows = df.shape[0]
cols_to_drop = [c for c in df.columns if df[c].isnull().sum() / n_rows > missing_threshold]
print("Columns to drop due to high missingness (>50%):", cols_to_drop)
df.drop(columns=cols_to_drop, inplace=True)

# 5) Drop duplicates
n_before = df.shape[0]
df.drop_duplicates(inplace=True)
n_after = df.shape[0]
print(f"Dropped {n_before - n_after} duplicate rows.")

# 6) Convert datatypes
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Numeric columns sample:", numeric_cols[:10])
print("Categorical columns sample:", cat_cols[:20])

# 7) Impute missing values
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Missing')

for col in df.select_dtypes(include=[np.number]).columns:
    if df[col].isnull().sum() > 0:
        df[col] = num_imputer.fit_transform(df[[col]])
        print(f"Imputed numeric column {col} with median.")

for col in df.select_dtypes(include=['object']).columns:
    if df[col].isnull().sum() > 0:
        df[col] = cat_imputer.fit_transform(df[[col]]).ravel()
        print(f"Imputed categorical column {col} with 'Missing'.")


# 8) Target class balance
print("Target distribution (readmit_30d):")
display(df['readmit_30d'].value_counts(normalize=True))

# 9) Reduce cardinality for rarely occurring categories
def reduce_cardinality(series, threshold=0.01):
    freqs = series.value_counts(normalize=True)
    categories_to_keep = freqs[freqs >= threshold].index
    return series.apply(lambda x: x if x in categories_to_keep else 'Other')

# Apply to selected high cardinality categorical columns
for col in ['diag_1', 'diag_2', 'diag_3']:
    if col in df.columns:
        df[col] = df[col].astype(str)
        df[col] = reduce_cardinality(df[col], threshold=0.01)
        print(f"Reduced cardinality for {col}")

# 10) Feature engineering
# - Combine counts of lab procedures and procedures into a high/low flag
if 'num_lab_procedures' in df.columns:
    df['lab_proc_high'] = (df['num_lab_procedures'] > df['num_lab_procedures'].median()).astype(int)
    print("Created lab_proc_high flag.")

# - group time_in_hospital into categories (short/medium/long)
if 'time_in_hospital' in df.columns:
    df['time_hosp_cat'] = pd.cut(df['time_in_hospital'], bins=[0,3,7,100], labels=['short','medium','long'])

# 11) Prepare list of features for modeling
if 'readmitted' in df.columns:
    df.drop(columns=['readmitted'], inplace=True)

print("Columns after preprocessing:", df.shape[1])
display(df.head())


Unique values in 'readmitted': ['NO' '>30' '<30']
Dropped column: encounter_id
Dropped column: patient_nbr


max_glu_serum    96420
A1Cresult        84748
dtype: int64

Replaced '?' with NaN in race
Replaced '?' with NaN in weight
Replaced '?' with NaN in payer_code
Replaced '?' with NaN in medical_specialty
Replaced '?' with NaN in diag_1
Replaced '?' with NaN in diag_2
Replaced '?' with NaN in diag_3


weight                      98569
max_glu_serum               96420
A1Cresult                   84748
medical_specialty           49949
payer_code                  40256
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
time_in_hospital                0
admission_source_id             0
num_procedures                  0
num_lab_procedures              0
admission_type_id               0
discharge_disposition_id        0
age                             0
gender                          0
number_inpatient                0
number_emergency                0
number_outpatient               0
num_medications                 0
number_diagnoses                0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

Columns to drop due to high missingness (>50%): ['weight', 'max_glu_serum', 'A1Cresult']
Dropped 0 duplicate rows.
Numeric columns sample: ['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient']
Categorical columns sample: ['race', 'gender', 'age', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose']
Imputed categorical column race with 'Missing'.
Imputed categorical column payer_code with 'Missing'.
Imputed categorical column medical_specialty with 'Missing'.
Imputed categorical column diag_1 with 'Missing'.
Imputed categorical column diag_2 with 'Missing'.
Imputed categorical column diag_3 with 'Missing'.
Target distribution (readmit_30d):


readmit_30d
0    0.888401
1    0.111599
Name: proportion, dtype: float64

Reduced cardinality for diag_1
Reduced cardinality for diag_2
Reduced cardinality for diag_3
Created lab_proc_high flag.
Columns after preprocessing: 47


,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmit_30d,lab_proc_high,time_hosp_cat
0,Caucasian,Female,[0-10),6,25,1,1,Missing,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,0,0,short
1,Caucasian,Female,[10-20),1,1,7,3,Missing,Missing,59,...,No,No,No,No,No,Ch,Yes,0,1,short
2,AfricanAmerican,Female,[20-30),1,1,7,2,Missing,Missing,11,...,No,No,No,No,No,No,Yes,0,0,short
3,Caucasian,Male,[30-40),1,1,7,2,Missing,Missing,44,...,No,No,No,No,No,Ch,Yes,0,0,short
4,Caucasian,Male,[40-50),1,1,7,1,Missing,Missing,51,...,No,No,No,No,No,Ch,Yes,0,1,short


In [11]:
output_path = "data/cleaned_diabetic_data.csv"
df.to_csv(output_path, index=False)